# Import library

In [1]:
import pandas as pd
import numpy as np

# Clean dataset

## Load data

In [15]:
def add_ext_to_str(df, col):
    if type(col) == str:
        col = [col]
    for col in col:
        df[col] = df[col].astype(str) + '_' + col
    return df

def fill_na_with_text(df, col, add_ext=False):
    if type(col) == str:
        col = [col]
    for col in col:
        if add_ext:
            df[col] = df[col].fillna('NO_'+col)
        else:
            df[col] = df[col].fillna('NO_INFO')
    return df

def merge_col(df, col, sep=','):
    return df[col].apply(lambda x: sep.join(x), axis=1)

def generate_int_id(dataframe, id_col_name):
    """
    Generate unique integer id for users, product and answers

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    id_col_name : String 
        New integer id's column name.
        
    Returns
    -------
    Dataframe
        Updated dataframe containing new id column 
    """
    new_dataframe=dataframe.assign(
        int_id_col_name=np.arange(len(dataframe))
        ).reset_index(drop=True)
    return new_dataframe.rename(columns={'int_id_col_name': id_col_name})

def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping 

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    features_name : List
        List of feature columns name avaiable in dataframe. 
        
    Returns
    -------
    List of all features for mapping 
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features

def create_features(dataframe, features_name, id_col_name):
    """
    Generate features that will be ready for feeding into lightfm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe which contains features
    features_name : List
        List of feature columns name avaiable in dataframe
    id_col_name: String
        Column name which contains id of the product or
        answer that the features will map to.
        There are two possible values for this variable.
        1. product_id_num
        2. user_id_num

    Returns
    -------
    Pandas Series
        A pandas series containing process features
        that are ready for feed into lightfm.
        The format of each value
        will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
        Ex. -> (1, ['military', 'army', '5'])
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features

In [17]:
df_product = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/product_raw.csv")
df_product = df_product[df_product['ACTIVE'] == 'Y']
df_product.drop(index = df_product[df_product['REVIEW_NO'] <= 5].index, inplace=True)
df_product.rename(columns={'SECTION_L1': 'CID', 'TYPE_ID': 'SCID'}, inplace=True)

df_cat = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/category.csv")
df_brand = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/brand_name.csv")

df_user = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/user.csv")
df_user.rename(columns={'ID': 'UID'}, inplace=True)

df_review = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/review.csv", low_memory=False)
df_review = df_review[df_review['ACTION'] == 1]

In [18]:
df_review = df_review[df_review['EID'].isin(df_product['EID']) & df_review['UID'].isin(df_user['UID'])].reset_index(drop=True)
df_product = df_product[df_product['EID'].isin(df_review['EID'])].reset_index(drop=True)
df_user = df_user[df_user['UID'].isin(df_review['UID'])].reset_index(drop=True)

## Product

In [19]:
# clean category field
row_scid_is_cid = df_product['SCID'].isin(df_cat[df_cat['DEPTH_LEVEL'] == 1]['ID'].values)
df_product.loc[row_scid_is_cid, 'CID'] = df_product.loc[row_scid_is_cid, 'SCID']

row_scid_is_mid = df_product['SCID'].isin(df_cat[df_cat['DEPTH_LEVEL'] == 2]['ID'].values)
df_product.loc[row_scid_is_mid, 'CID'] = df_product.loc[row_scid_is_mid, 'SECTION_L2']

row_scid_notin_scid = ~(df_product['SCID'].isin(df_cat[df_cat['DEPTH_LEVEL'] == 3]['ID'].values))
df_product.loc[row_scid_notin_scid, 'SCID'] = -1
df_product.loc[df_product['CID'].isnull(), 'CID'] = -1
df_product['BID'].fillna(-1, inplace=True)
df_product['TAGS'].fillna('NO_TAGS', inplace=True)
df_product['TAGS_NAME'].fillna('NO_TAGS_NAME', inplace=True)

df_product['SCID'] = df_product['SCID'].astype(int)
df_product['CID'] = df_product['CID'].astype(int)
df_product['BID'] = df_product['BID'].astype(int)
df_product['TAGS'] = df_product['TAGS'].astype(str)
df_product['TAGS_NAME'] = df_product['TAGS_NAME'].astype(str)

In [20]:
# fillna
df_product.loc[df_product['SCID'] == -1, 'SCID'] = 'NO_SUBCATEGORY'
df_product.loc[df_product['SCID'] == -1, 'CID'] = 'NO_CATEGORY'
df_product.loc[df_product['BID'] == -1, 'BID'] = 'NO_INFO'

# add extension to NaN value
df_product['BID'] = 'BID_' + df_product['BID'].astype(str)
df_product['SCID'] = 'SCID_' + df_product['SCID'].astype(str)
df_product['CID'] = 'CID_' + df_product['CID'].astype(str)

# create RATE_TEXT
for lw_bound in np.arange(0, 5, 0.5):
    up_bound = lw_bound + 0.5
    df_product.loc[(df_product['RATE'] >= lw_bound) & (df_product['RATE'] < up_bound), 'RATE_TEXT'] = str(lw_bound) + ' - ' + str(up_bound)
df_product['RATE_TEXT'].fillna('NO_RATE', inplace=True)

# join tags
df_product['TAGS'] = df_product['TAGS'].apply(lambda tags: ','.join(tag.strip() for tag in tags.split(',')))
df_product['TAGS_NAME'] = df_product['TAGS_NAME'].apply(lambda tags: ','.join(tag.strip() for tag in tags.split(',')))
df_product['product_tag'] = merge_col(df_product, ['BID', 'SCID', 'CID', 'TAGS', 'TAGS_NAME', 'RATE_TEXT'])
df_product = generate_int_id(df_product, 'product_id_num')
df_product['product_tag'] = (
    df_product['product_tag'].str.split(',').apply(set).str.join(','))

df_product = df_product[['EID', 'PRODUCT_NAME', 'product_tag', 'product_id_num']]
df_product.reset_index(drop=True, inplace=True)
df_product.head(1)

,EID,PRODUCT_NAME,product_tag,product_id_num
0,13001,UV Aqua Rich Watery Essence SPF50+ PA++++,"ครีมกันแดด,กันแดด,Facial Sunscreen,sun care,BI...",0


## User

In [21]:
# clean 'AGE_TEXT'
df_user.loc[df_user['AGE'] <= 11, 'AGE_TEXT'] = np.nan
df_user.loc[df_user['BIRTHDAY'] <= 11, 'AGE_TEXT'] = np.nan
df_user.loc[(df_user['AGE'] < 1900) & (df_user['AGE'] > 100), 'AGE_TEXT'] = np.nan
df_user.loc[(df_user['BIRTHDAY'] > 50) & (df_user['BIRTHDAY'] < 100), 'AGE_TEXT'] = np.nan

df_user.loc[(df_user['BIRTHDAY'] > 11) & (df_user['BIRTHDAY'] < 18), 'AGE_TEXT'] = 'ตำกว่า 18'
df_user.loc[(df_user['BIRTHDAY'] >= 18) & (df_user['BIRTHDAY'] < 25), 'AGE_TEXT'] = '18 - 24'
df_user.loc[(df_user['BIRTHDAY'] >= 25) & (df_user['BIRTHDAY'] < 31), 'AGE_TEXT'] = '25 - 30'
df_user.loc[(df_user['BIRTHDAY'] >= 31) & (df_user['BIRTHDAY'] < 36), 'AGE_TEXT'] = '31 - 35'
df_user.loc[(df_user['BIRTHDAY'] >= 36) & (df_user['BIRTHDAY'] < 41), 'AGE_TEXT'] = '36 - 40'
df_user.loc[(df_user['BIRTHDAY'] >= 41) & (df_user['BIRTHDAY'] < 50), 'AGE_TEXT'] = '40 ขึ้นไป'

# fill na with text
df_user = fill_na_with_text(df_user, ['UF_SKINFACE', 'UF_BODYTYPE'])
df_user = add_ext_to_str(df_user, ['UF_SKINFACE', 'UF_BODYTYPE'])
df_user = fill_na_with_text(df_user, ['AGE_TEXT', 'PERSONAL_GENDER', 'UF_HAIRTYPE'], add_ext=True)

# clean 'PERSONAL_GENDER'
is_female = np.isin(df_user['PERSONAL_GENDER'], 'Female')
is_male = np.isin(df_user['PERSONAL_GENDER'], 'Male')
is_nan = np.isin(df_user['PERSONAL_GENDER'], 'NO_PERSONAL_GENDER')
df_user.loc[~is_female & ~is_male & ~is_nan, 'PERSONAL_GENDER'] = 'LGBTQ+'

# create 'user_tag' to merge all tags
df_user['user_tag'] = merge_col(df_user, ['AGE_TEXT', 'PERSONAL_GENDER', 'UF_SKINFACE', 'UF_BODYTYPE', 'UF_HAIRTYPE'])
df_user = generate_int_id(df_user, 'user_id_num')

df_user = df_user[['UID', 'user_tag', 'user_id_num']].reset_index(drop=True)
df_user.head(1)

,UID,user_tag,user_id_num
0,4,"40 ขึ้นไป,Male,ผิวมัน_UF_SKINFACE,NO_INFO_UF_B...",0


## Merge

In [22]:
df_merge = df_review.merge(df_product[['EID', 'product_tag', 'product_id_num']], how='inner',
    left_on='EID', right_on='EID')
df_merge = df_merge.merge(df_user[['UID', 'user_tag', 'user_id_num']], how='inner',
    left_on='UID', right_on='UID')
df_merge.head(1)

,ID,EID,BID,ACTION,COMMENT,REGULAR,ACTUAL,DATE_CREATE,modify_date,TIMESTAMP_TOP,...,LOCATION,EFFECT,D_USER,FID,campaing_name,SEARCH_TXT,product_tag,product_id_num,user_tag,user_id_num
0,161561,40039,3081,1,ได้รับสี 09 Honey Brown สีน้ำตาลทอง ขนาดกำลังด...,NO,0.0,2023-08-23 15:10:32.000,2023-08-23 15:10:32.000,2023-08-23 15:08:57.000,...,กิจกรรม cosmenet.in.th,NaN,NaN,NaN,product_meilinda_40039,ได้รับสี 09 Honey Brown สีน้ำตาลทอง ขนาดกำลังด...,"ดินสอเขียนคิ้วเนื้อเนียน,ดินสอเขียนคิ้ว,เขียนค...",1243,"25 - 30,Female,ผิวผสม_UF_SKINFACE,ผิวธรรมดา_UF...",13946


In [23]:
user_prev_product_tags = df_merge[['UID', 'product_tag']]
user_prev_product_tags = user_prev_product_tags.dropna()
user_prev_product_tags = user_prev_product_tags.groupby(
    ['UID'])['product_tag'].apply(
        ','.join).reset_index()

# drop duplicates tags from each user rows
user_prev_product_tags['product_tag'] = (
    user_prev_product_tags['product_tag'].str.split(',').apply(set).str.join(','))

# finally merge the dataframe with user dataframe 
df_user = df_user.merge(user_prev_product_tags, how='left', on='UID')

# join user tags and their answered tags 
# we replace nan values with ""
df_user['UID_all_tags'] = (
    df_user[['user_tag', 'product_tag']].apply(
        lambda x: ','.join(x.dropna()),
        axis=1))

df_user.head(1)

,UID,user_tag,user_id_num,product_tag,UID_all_tags
0,4,"40 ขึ้นไป,Male,ผิวมัน_UF_SKINFACE,NO_INFO_UF_B...",0,"เขียนขอบปาก,เอสเซ้นส์ทาหน้า,Essence,dove inten...","40 ขึ้นไป,Male,ผิวมัน_UF_SKINFACE,NO_INFO_UF_B..."


In [24]:
df_product['product_tag'] = df_product['product_tag'].str.split(',').apply(set).str.join(',')
df_user['UID_all_tags'] = df_user['UID_all_tags'].str.split(',').apply(set).str.join(',')

In [25]:
product_feature_list = generate_feature_list(
    df_product,
    ['product_tag'])

user_feature_list = generate_feature_list(
    df_user,
    ['UID_all_tags'])

In [26]:
df_product['product_features'] = create_features(
    df_product, ['product_tag'], 
    'product_id_num')

df_user['user_features'] = create_features(
    df_user,
    ['UID_all_tags'],
    'user_id_num')

# LightFM

## Import

In [28]:
from lightfm.data import Dataset
from lightfm import LightFM

from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k
from lightfm.cross_validation import random_train_test_split

## Build Dataset

In [29]:
dataset = Dataset()
dataset.fit(
    set(df_user['user_id_num']), 
    set(df_product['product_id_num']),
    item_features=product_feature_list, 
    user_features=user_feature_list)

In [30]:
df_merge['point_tuple'] = list(zip(
    df_merge.user_id_num, df_merge.product_id_num, df_merge.POINT))

In [31]:
interactions, weights = dataset.build_interactions(
    df_merge['point_tuple'])

In [32]:
product_features = dataset.build_item_features(
    df_product['product_features'])

user_features = dataset.build_user_features(
    df_user['user_features'])

In [33]:
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.2)

## Train

In [75]:
model = LightFM(
    no_components=150,
    learning_rate=0.05,
    loss='warp',
    random_state=2019)

model.fit(
    interactions,
    item_features=product_features,
    user_features=user_features, sample_weight=weights,
    epochs=20, num_threads=4, verbose=True)

Epoch: 100%|██████████| 20/20 [04:19<00:00, 12.99s/it]


In [34]:
from skopt import forest_minimize

In [35]:
def objective(params):
    print('=====================================')
    # unpack
    epochs, learning_rate,\
    no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    print(params)
    model = LightFM(loss='warp',
                    random_state=2016,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(train_interactions, epochs=epochs,
              num_threads=8, verbose=True)
    
    patks = precision_at_k(model, test_interactions, train_interactions=None,
                           k=5, num_threads=8).mean()
    # Make negative because we want to _minimize_ objective
    out = -patks
    # Handle some weird numerical shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [36]:
space = [(1, 260), # epochs
         (10**-4, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-6, 10**-1, 'log-uniform'), # alpha
        ]

res_fm = forest_minimize(objective, space, n_calls=250,
                     random_state=0,
                     verbose=True)

Iteration No: 1 started. Evaluating function at random point.
[48, 0.07257005721594281, 87, 0.019486241836466404]


Epoch:   0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 100%|██████████| 48/48 [00:07<00:00,  6.24it/s]


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 9.5057
Function value obtained: -0.0030
Current minimum: -0.0030
Iteration No: 2 started. Evaluating function at random point.
[10, 0.031207198770106446, 56, 3.073781785362612e-05]


Epoch: 100%|██████████| 10/10 [00:00<00:00, 11.50it/s]


...

Iteration No: 248 ended. Search finished for the next optimal point.
Time taken: 10.9065
Function value obtained: -0.0392
Current minimum: -0.0503
Iteration No: 249 started. Searching for the next optimal point.
[141, 0.06754346250596092, 89, 0.0018949520186317926]


Epoch: 100%|██████████| 141/141 [00:20<00:00,  6.91it/s]


Iteration No: 249 ended. Search finished for the next optimal point.
Time taken: 22.5476
Function value obtained: -0.0226
Current minimum: -0.0503
Iteration No: 250 started. Searching for the next optimal point.
[75, 0.06911528281384702, 44, 0.001172587657499896]


Epoch: 100%|██████████| 75/75 [00:06<00:00, 11.68it/s]


Iteration No: 250 ended. Search finished for the next optimal point.
Time taken: 7.7115
Function value obtained: -0.0441
Current minimum: -0.0503


In [37]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'alpha']
for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))
print(f'params : {res_fm.x}')

Maximimum p@k found: 0.05031
Optimal parameters:
epochs: 250
learning_rate: 0.05489768662585542
no_components: 165
alpha: 0.0010024970478760666
params : [250, 0.05489768662585542, 165, 0.0010024970478760666]


In [38]:
def objective_wsideinfo(params):
    # unpack
    print('=====================================')
    print(params)
    epochs, learning_rate,\
    no_components, item_alpha,\
    scale = params
    
    user_alpha = item_alpha * scale
    model = LightFM(loss='warp',
                    random_state=2016,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(train_interactions, epochs=epochs,
              item_features=product_features, user_features=user_features, 
              num_threads=8, verbose=True)
    
    patks = precision_at_k(model, test_interactions, item_features=product_features, user_features=user_features,
                           check_intersections=False , k=5, num_threads=8).mean()
    # Make negative because we want to _minimize_ objective
    out = -patks
    # Weird shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [39]:
res_fm.x.append(1.)
res_fm.x

[250, 0.05489768662585542, 165, 0.0010024970478760666, 1.0]

In [40]:
space = [(1, 260), # epochs
         (10**-3, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-5, 10**-3, 'log-uniform'), # item_alpha
         (0.001, 1., 'log-uniform') # user_scaling
        ]
x0 = res_fm.x
# This typecast is required
res_fm_itemfeat = forest_minimize(objective_wsideinfo, space, n_calls=50,
                                  x0=x0,
                                  random_state=0,
                                  verbose=True)

Iteration No: 1 started. Evaluating function at provided point.
[250, 0.05489768662585542, 165, 0.0010024970478760666, 1.0]


Epoch:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch:   2%|▏         | 5/250 [08:13<6:41:42, 98.38s/it] 

In [33]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm_itemfeat.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'item_alpha', 'scaling']
for (p, x_) in zip(params, res_fm_itemfeat.x):
    print('{}: {}'.format(p, x_))

NameError: name 'res_fm_itemfeat' is not defined

In [20]:
from tqdm.notebook import tqdm

In [22]:
import pickle
import os
file = 'weights/lightFM_model_tune.pickle'

In [25]:
params = [148, 0.23621062223458383, 192, 2.8247736497887107e-05, 0.9739410842936009]
epochs, learning_rate, no_components, item_alpha, scale = params

adagrad_auc = []
max_score = 0
model = LightFM(no_components=no_components,
                        loss='warp',
                        learning_rate=learning_rate,
                        user_alpha=item_alpha * scale,
                        item_alpha=item_alpha
                        )

for epoch in tqdm(range(epochs)):
    model.fit_partial(train_interactions, item_features=product_features,
                        user_features=user_features, num_threads=8, epochs=1)
    # score = auc_score( 
    #     adagrad_model, interactions, 
    #     item_features=product_features, 
    #     user_features=user_features, 
    #     num_threads=8).mean()
    test_score = precision_at_k(model, test_interactions, item_features=product_features, user_features=user_features, train_interactions=train_interactions,
                                num_threads=8, check_intersections=False, k=5).mean()
    if test_score > max_score:
        max_score = test_score
        with open(file, 'wb') as fle:
            pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)
        print('test score :', test_score)
        score = precision_at_k(model, train_interactions, item_features=product_features, user_features=user_features, k=5, 
                                    num_threads=8).mean()
        print(epoch, score)
        print("=====================================")

  0%|          | 0/148 [00:00<?, ?it/s]

0 0.14994614
test score : 0.090493314
1 0.19145761
test score : 0.11682458
2 0.20940436
test score : 0.127246
3 0.218978
test score : 0.1297942
4 0.23001386
test score : 0.13593598
5 0.24006465
test score : 0.14263314
6 0.24582115
test score : 0.14619407
7 0.25006926
test score : 0.14861158
8 0.2547176
test score : 0.14998367
9 0.25750345
test score : 0.15174781
10 0.25981224
test score : 0.15269521
11 0.2622595
test score : 0.154296
12 0.2661998
test score : 0.15648481
13 0.2691704
test score : 0.15749756
14 0.27058643
test score : 0.16027443
16 0.27569652
test score : 0.16099313
18 0.27783594
test score : 0.16324733
20 0.28206864
test score : 0.16452141
21 0.2826997
test score : 0.16471744
22 0.2846083
test score : 0.1657955
23 0.28713253
test score : 0.16625287
24 0.2877174
test score : 0.168017
26 0.29004157
test score : 0.16922574
28 0.29175007
test score : 0.17049985
32 0.2963214
test score : 0.17098989
33 0.29701403
test score : 0.17239466
34 0.29863015
test score : 0.17357074
4

In [30]:
model.learning_rate = 0.05

In [29]:
for epoch in tqdm(range(100)):
    model.fit_partial(train_interactions, item_features=product_features,
                        user_features=user_features, num_threads=8, epochs=1)
    # score = auc_score( 
    #     adagrad_model, interactions, 
    #     item_features=product_features, 
    #     user_features=user_features, 
    #     num_threads=8).mean()
    test_score = precision_at_k(model, test_interactions, item_features=product_features, user_features=user_features, train_interactions=train_interactions,
                                num_threads=8, check_intersections=False, k=5).mean()
    if test_score > max_score:
        max_score = test_score
        with open(file, 'wb') as fle:
            pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)
        print('test score :', test_score)
        score = precision_at_k(model, train_interactions, item_features=product_features, user_features=user_features, k=5, 
                                    num_threads=8).mean()
        print(epoch, score)
        print("=====================================")

  0%|          | 0/50 [00:00<?, ?it/s]

test score : 0.19297618
1 0.3614745
test score : 0.19366221
6 0.36168998


In [25]:
for n, i in enumerate(adagrad_auc):
    if i == max(adagrad_auc):
        print(n, i)

19 0.17044514


In [20]:
import pickle
import os
file = 'weights/lightFM_model_tune_lr0.05.pickle'
if not os.path.exists(file):
    with open(file, 'wb') as fle:
        pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    print(f"{file} exists.")

In [31]:
import pickle
loaded_model = pickle.load(open('/home/music/Desktop/measure_model/weights/lightFM_model_tune.pickle', 'rb'))

In [32]:
loaded_model.learning_rate = 0.01
file = 'weights/lightFM_model_tune_lr0.05.pickle'

In [35]:
for epoch in tqdm(range(100)):
    loaded_model.fit_partial(train_interactions, item_features=product_features,
                        user_features=user_features, num_threads=8, epochs=1)
    # score = auc_score( 
    #     adagrad_model, interactions, 
    #     item_features=product_features, 
    #     user_features=user_features, 
    #     num_threads=8).mean()
    test_score = precision_at_k(loaded_model, test_interactions, item_features=product_features, user_features=user_features, train_interactions=train_interactions,
                                num_threads=8, check_intersections=False, k=5).mean()
    if test_score > max_score:
        max_score = test_score
        with open(file, 'wb') as fle:
            pickle.dump(loaded_model, fle, protocol=pickle.HIGHEST_PROTOCOL)
        print('test score :', test_score)
        score = precision_at_k(loaded_model, train_interactions, item_features=product_features, user_features=user_features, k=5, 
                                    num_threads=8).mean()
        print(epoch, score)
        print("=====================================")

  0%|          | 0/100 [00:00<?, ?it/s]

test score : 0.19500165
12 0.36601508


In [76]:
def calculate_auc_score(lightfm_model, interactions_matrix, 
                        product_features, User_features): 
    """
    Measure the ROC AUC metric for a model. 
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model 
        A fitted lightfm model 
    interactions_matrix : 
        A lightfm interactions matrix 
    product_features, User_features: 
        Lightfm features 
        
    Returns
    -------
    String containing AUC score 
    """
    score = auc_score( 
        lightfm_model, interactions_matrix, 
        item_features=product_features, 
        user_features=User_features, 
        num_threads=8).mean()
    return score

In [79]:
calculate_auc_score(model, interactions, product_features, user_features)

0.99271107

In [83]:
precision_at_k(model, interactions, item_features=product_features, user_features=user_features, k=1, 
                                   num_threads=8, check_intersections=False).mean()

0.29803705

In [84]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def previous_product(user, n_previous=3, show=True):
    # print their previous answered product title
    previous_product_id_num = df_merge.loc[df_merge['user_id_num'] == user][:n_previous]['product_id_num']
    df_previous_product = df_product.loc[df_product['product_id_num'].isin(previous_product_id_num)]
    if show:
        print('User Id (' + str(user) + "): Previous review product")
        display_side_by_side(
            df_previous_product[['EID', 'PRODUCT_NAME', 'product_features']],
            df_user.loc[df_user.user_id_num == user][['UID', 'user_id_num','user_tag']])
    return df_previous_product

def recommend_product(model, user, n_previous=3, top_n=10, show=True):
    df_previous_product = previous_product(user, n_previous, show=show)
    
    # predict
    discard_product_id = df_previous_product['product_id_num'].values.tolist()
    df_use_for_prediction = df_product.loc[~df_product['product_id_num'].isin(discard_product_id)]
    product_id_for_predict = df_use_for_prediction['product_id_num'].values.tolist()
    
    scores = model.predict(
        user,
        product_id_for_predict,
        item_features=product_features,
        user_features=user_features)
    
    df_use_for_prediction = df_use_for_prediction.assign(scores=scores)
    df_use_for_prediction = df_use_for_prediction.sort_values(by='scores', ascending=False)[:top_n]
    if show:
        print('User Id (' + str(user) + "): Recommended product: ")
        display(df_use_for_prediction[['EID', 'PRODUCT_NAME', 'product_features']])
    return df_use_for_prediction

def recommends_product(model, user_ids, n_previous=3, top_n=10):
    for user in user_ids:
        recommend_product(model, user, n_previous, top_n)
        print("=========================================================================================================")

In [85]:
from tqdm.notebook import tqdm

In [92]:
n_previous=5
count = 0
# num_users = 1000
num_users = df_merge['user_id_num'].max()

for i in tqdm(range(num_users)):
    previous = previous_product(i, None, show=False)
    recommend = recommend_product(model, i, n_previous=n_previous, top_n=10, show=False)
    result = recommend[recommend['EID'].isin(previous['EID'])]
    
    if len(result) > 0:
        count += 1
        # print('User Id (' + str(i) + "): Previous review product")
        # display_side_by_side(
        #     previous[['EID', 'PRODUCT_NAME']],
        #     previous_product(i, 5, show=False)[['EID', 'PRODUCT_NAME']],
        #     recommend[['EID', 'PRODUCT_NAME']],
        #     result[['EID', 'PRODUCT_NAME']])
        # print("=========================================================================================================")
print(count)
print(count/num_users*100)

  0%|          | 0/14467 [00:00<?, ?it/s]

1869
12.91905716458146


In [31]:
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.2)

In [33]:
train_weights, test_weights = random_train_test_split(weights, test_percentage=0.2)


In [60]:
model = LightFM(
    no_components=250,
    learning_rate=0.0005,
    loss='warp',
    random_state=2019)

model.fit(
    train_interactions,
    item_features=product_features,
    user_features=user_features,
    epochs=20, num_threads=8, verbose=True)

Epoch: 100%|██████████| 20/20 [05:20<00:00, 16.04s/it]


In [61]:
print("Test precision_at_k score:", precision_at_k(loaded_model, interactions, item_features=product_features, user_features=user_features, train_interactions=train_interactions,
                                   num_threads=8, check_intersections=False).mean())

Test precision_at_k score: 0.13128889


In [ ]:
0.13, 150


In [43]:
print("Test AUC score:", auc_score(model, test_interactions, item_features=product_features, user_features=user_features, 
                                   num_threads=7).mean())

Test AUC score: 0.98318493


In [45]:
test_auc = auc_score(model, test_interactions, item_features=product_features, user_features=user_features,
                     train_interactions=train_interactions, num_threads=7, check_intersections=False).mean()
print('Collaborative filtering test AUC: %s' % test_auc)

Collaborative filtering test AUC: 0.98344797
